In [1]:
import pandas as pd
import seaborn as sns

In [2]:
desc = pd.read_csv('data/VariableDefinitions.csv')
train = pd.read_csv('data/Train.csv')
test = pd.read_csv('data/Test.csv')
sub = pd.read_csv('data/Samplesubmission.csv')

In [6]:
df.loc[1].values

array([nan,
       'Le dataset churn comprend 19 variables dont 15 variables numeriques et 04 variables categoriques.',
       'The churn dataset includes 19 variables including 15 numeric variables and 04 categorical variables.'],
      dtype=object)

In [22]:
desc.drop(['Unnamed: 1'], axis=1, inplace=True)

In [23]:
desc.iloc[2:, ].values

array([['user_id', nan],
       ['REGION', 'the location of each client'],
       ['TENURE', 'duration in the network'],
       ['MONTANT', 'top-up amount'],
       ['FREQUENCE_RECH', '\xa0number of times the customer refilled'],
       ['REVENUE', 'monthly income of each client'],
       ['ARPU_SEGMENT', 'income over 90 days / 3'],
       ['FREQUENCE', 'number of times the client has made an income'],
       ['DATA_VOLUME', 'number of connections'],
       ['ON_NET', 'inter expresso call'],
       ['ORANGE', 'call to orange'],
       ['TIGO', 'call to Tigo'],
       ['ZONE1', 'call to zones1'],
       ['ZONE2', 'call to zones2'],
       ['MRG', 'a client who is going'],
       ['REGULARITY', 'number of times the client is active for 90 days'],
       ['TOP_PACK', 'the most active packs'],
       ['FREQ_TOP_PACK',
        'number of times the client has activated the top pack packages'],
       ['CHURN', 'variable to predict - Target']], dtype=object)

In [25]:
train.head(2)

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,FATICK,K > 24 month,4250.0,15.0,4251.0,1417.0,17.0,4.0,388.0,46.0,1.0,1.0,2.0,NO,54,On net 200F=Unlimited _call24H,8.0,0
1,00000cb4a5d760de88fecb38e2f71b7bec52e834,NaN,I 18-21 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,4,NaN,NaN,1


In [27]:
train.dtypes

user_id            object
REGION             object
TENURE             object
MONTANT           float64
FREQUENCE_RECH    float64
REVENUE           float64
ARPU_SEGMENT      float64
FREQUENCE         float64
DATA_VOLUME       float64
ON_NET            float64
ORANGE            float64
TIGO              float64
ZONE1             float64
ZONE2             float64
MRG                object
REGULARITY          int64
TOP_PACK           object
FREQ_TOP_PACK     float64
CHURN               int64
dtype: object

In [26]:
train.isna().sum()

user_id                 0
REGION             849299
TENURE                  0
MONTANT            756739
FREQUENCE_RECH     756739
REVENUE            726048
ARPU_SEGMENT       726048
FREQUENCE          726048
DATA_VOLUME       1060433
ON_NET             786675
ORANGE             895248
TIGO              1290016
ZONE1             1984327
ZONE2             2017224
MRG                     0
REGULARITY              0
TOP_PACK           902594
FREQ_TOP_PACK      902594
CHURN                   0
dtype: int64

In [45]:
test.isna().sum()

user_id                0
REGION            149728
TENURE                 0
MONTANT           133055
FREQUENCE_RECH    133055
REVENUE           127373
ARPU_SEGMENT      127373
FREQUENCE         127373
DATA_VOLUME       187040
ON_NET            138514
ORANGE            157230
TIGO              227001
ZONE1             350266
ZONE2             356051
MRG                    0
REGULARITY             0
TOP_PACK          158779
FREQ_TOP_PACK     158779
is_train               0
dtype: int64

In [3]:
train['is_train'] = True
test['is_train'] = False

full = pd.concat([train.drop('CHURN', axis=1), test], ignore_index=True)

In [4]:
for col in full.columns:

    if full[col].isna().sum() == 0:
        continue

    full[col + '_isna'] = full[col].isna().astype('int')
    filler = train[col].mode()[0]

    if train[col].dtype != 'object':
        filler = train[col].median()
    
    full[col].fillna(filler, inplace=True)

full = full.drop('MRG', axis=1)

freq_map = train['TOP_PACK'].value_counts().to_dict()
full['TOP_PACK'] = full['TOP_PACK'].map(freq_map)
full.fillna(0, inplace=True)

In [70]:
full.isna().sum()

user_id                0
REGION                 0
TENURE                 0
MONTANT                0
FREQUENCE_RECH         0
REVENUE                0
ARPU_SEGMENT           0
FREQUENCE              0
DATA_VOLUME            0
ON_NET                 0
ORANGE                 0
TIGO                   0
ZONE1                  0
ZONE2                  0
REGULARITY             0
TOP_PACK               0
FREQ_TOP_PACK          0
is_train               0
REGION_isna            0
MONTANT_isna           0
FREQUENCE_RECH_isna    0
REVENUE_isna           0
ARPU_SEGMENT_isna      0
FREQUENCE_isna         0
DATA_VOLUME_isna       0
ON_NET_isna            0
ORANGE_isna            0
TIGO_isna              0
ZONE1_isna             0
ZONE2_isna             0
TOP_PACK_isna          0
FREQ_TOP_PACK_isna     0
dtype: int64

In [5]:
full = pd.get_dummies(full.drop('user_id', axis=1))

In [6]:
train, target = full[full['is_train']], train['CHURN']

In [65]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

In [8]:
X_train, X_test, y_train, y_test = train_test_split(train, target)

In [47]:
lgbm = LGBMClassifier(learning_rate=0.075)

lgbm.fit(X_train, y_train)

LGBMClassifier(learning_rate=0.075)

In [48]:
roc_auc_score(y_test, lgbm.predict_proba(X_test)[:, 1])

0.9310374588982292

In [15]:
cbc = CatBoostClassifier()

cbc.fit(X_train, y_train)

Learning rate set to 0.241482
0:	learn: 0.4062262	total: 470ms	remaining: 7m 49s
1:	learn: 0.3155420	total: 808ms	remaining: 6m 43s
2:	learn: 0.2832153	total: 1.09s	remaining: 6m 1s
3:	learn: 0.2685991	total: 1.36s	remaining: 5m 39s
4:	learn: 0.2620563	total: 1.63s	remaining: 5m 24s
5:	learn: 0.2581516	total: 1.92s	remaining: 5m 18s
6:	learn: 0.2560946	total: 2.18s	remaining: 5m 9s
7:	learn: 0.2550714	total: 2.43s	remaining: 5m 1s
8:	learn: 0.2544022	total: 2.74s	remaining: 5m 1s
9:	learn: 0.2539632	total: 3.01s	remaining: 4m 57s
10:	learn: 0.2534264	total: 3.31s	remaining: 4m 57s
11:	learn: 0.2530723	total: 3.59s	remaining: 4m 55s
12:	learn: 0.2529483	total: 3.95s	remaining: 4m 59s
13:	learn: 0.2528528	total: 4.25s	remaining: 4m 59s
14:	learn: 0.2527004	total: 4.54s	remaining: 4m 58s
15:	learn: 0.2526301	total: 4.86s	remaining: 4m 58s
16:	learn: 0.2525172	total: 5.16s	remaining: 4m 58s
17:	learn: 0.2524322	total: 5.46s	remaining: 4m 57s
18:	learn: 0.2523626	total: 5.74s	remaining: 4m 

158:	learn: 0.2499997	total: 50.9s	remaining: 4m 29s
159:	learn: 0.2499936	total: 51.2s	remaining: 4m 28s
160:	learn: 0.2499845	total: 51.5s	remaining: 4m 28s
161:	learn: 0.2499771	total: 51.9s	remaining: 4m 28s
162:	learn: 0.2499651	total: 52.2s	remaining: 4m 28s
163:	learn: 0.2499564	total: 52.5s	remaining: 4m 27s
164:	learn: 0.2499444	total: 52.8s	remaining: 4m 27s
165:	learn: 0.2499365	total: 53.1s	remaining: 4m 26s
166:	learn: 0.2499291	total: 53.4s	remaining: 4m 26s
167:	learn: 0.2499164	total: 53.7s	remaining: 4m 26s
168:	learn: 0.2499087	total: 54s	remaining: 4m 25s
169:	learn: 0.2498997	total: 54.3s	remaining: 4m 25s
170:	learn: 0.2498889	total: 54.6s	remaining: 4m 24s
171:	learn: 0.2498800	total: 54.9s	remaining: 4m 24s
172:	learn: 0.2498743	total: 55.2s	remaining: 4m 23s
173:	learn: 0.2498638	total: 55.5s	remaining: 4m 23s
174:	learn: 0.2498511	total: 55.9s	remaining: 4m 23s
175:	learn: 0.2498421	total: 56.2s	remaining: 4m 23s
176:	learn: 0.2498364	total: 56.5s	remaining: 4m

312:	learn: 0.2488558	total: 1m 38s	remaining: 3m 35s
313:	learn: 0.2488484	total: 1m 38s	remaining: 3m 35s
314:	learn: 0.2488401	total: 1m 38s	remaining: 3m 35s
315:	learn: 0.2488350	total: 1m 39s	remaining: 3m 34s
316:	learn: 0.2488283	total: 1m 39s	remaining: 3m 34s
317:	learn: 0.2488173	total: 1m 39s	remaining: 3m 34s
318:	learn: 0.2488147	total: 1m 40s	remaining: 3m 33s
319:	learn: 0.2488095	total: 1m 40s	remaining: 3m 33s
320:	learn: 0.2488008	total: 1m 40s	remaining: 3m 33s
321:	learn: 0.2487917	total: 1m 41s	remaining: 3m 32s
322:	learn: 0.2487830	total: 1m 41s	remaining: 3m 32s
323:	learn: 0.2487781	total: 1m 41s	remaining: 3m 32s
324:	learn: 0.2487727	total: 1m 41s	remaining: 3m 31s
325:	learn: 0.2487656	total: 1m 42s	remaining: 3m 31s
326:	learn: 0.2487612	total: 1m 42s	remaining: 3m 31s
327:	learn: 0.2487549	total: 1m 42s	remaining: 3m 30s
328:	learn: 0.2487492	total: 1m 43s	remaining: 3m 30s
329:	learn: 0.2487463	total: 1m 43s	remaining: 3m 30s
330:	learn: 0.2487384	total:

466:	learn: 0.2479317	total: 2m 25s	remaining: 2m 46s
467:	learn: 0.2479258	total: 2m 26s	remaining: 2m 45s
468:	learn: 0.2479228	total: 2m 26s	remaining: 2m 45s
469:	learn: 0.2479190	total: 2m 26s	remaining: 2m 45s
470:	learn: 0.2479149	total: 2m 26s	remaining: 2m 44s
471:	learn: 0.2479109	total: 2m 27s	remaining: 2m 44s
472:	learn: 0.2479033	total: 2m 27s	remaining: 2m 44s
473:	learn: 0.2478968	total: 2m 27s	remaining: 2m 43s
474:	learn: 0.2478895	total: 2m 28s	remaining: 2m 43s
475:	learn: 0.2478829	total: 2m 28s	remaining: 2m 43s
476:	learn: 0.2478769	total: 2m 28s	remaining: 2m 43s
477:	learn: 0.2478709	total: 2m 29s	remaining: 2m 42s
478:	learn: 0.2478650	total: 2m 29s	remaining: 2m 42s
479:	learn: 0.2478584	total: 2m 29s	remaining: 2m 42s
480:	learn: 0.2478523	total: 2m 29s	remaining: 2m 41s
481:	learn: 0.2478483	total: 2m 30s	remaining: 2m 41s
482:	learn: 0.2478426	total: 2m 30s	remaining: 2m 41s
483:	learn: 0.2478363	total: 2m 30s	remaining: 2m 40s
484:	learn: 0.2478325	total:

620:	learn: 0.2470735	total: 3m 12s	remaining: 1m 57s
621:	learn: 0.2470696	total: 3m 12s	remaining: 1m 56s
622:	learn: 0.2470645	total: 3m 12s	remaining: 1m 56s
623:	learn: 0.2470601	total: 3m 13s	remaining: 1m 56s
624:	learn: 0.2470546	total: 3m 13s	remaining: 1m 55s
625:	learn: 0.2470483	total: 3m 13s	remaining: 1m 55s
626:	learn: 0.2470438	total: 3m 13s	remaining: 1m 55s
627:	learn: 0.2470373	total: 3m 14s	remaining: 1m 55s
628:	learn: 0.2470351	total: 3m 14s	remaining: 1m 54s
629:	learn: 0.2470273	total: 3m 14s	remaining: 1m 54s
630:	learn: 0.2470238	total: 3m 15s	remaining: 1m 54s
631:	learn: 0.2470173	total: 3m 15s	remaining: 1m 53s
632:	learn: 0.2470126	total: 3m 15s	remaining: 1m 53s
633:	learn: 0.2470070	total: 3m 16s	remaining: 1m 53s
634:	learn: 0.2470015	total: 3m 16s	remaining: 1m 52s
635:	learn: 0.2469995	total: 3m 16s	remaining: 1m 52s
636:	learn: 0.2469957	total: 3m 16s	remaining: 1m 52s
637:	learn: 0.2469901	total: 3m 17s	remaining: 1m 51s
638:	learn: 0.2469849	total:

772:	learn: 0.2463187	total: 3m 56s	remaining: 1m 9s
773:	learn: 0.2463151	total: 3m 57s	remaining: 1m 9s
774:	learn: 0.2463102	total: 3m 57s	remaining: 1m 8s
775:	learn: 0.2463031	total: 3m 57s	remaining: 1m 8s
776:	learn: 0.2462964	total: 3m 58s	remaining: 1m 8s
777:	learn: 0.2462876	total: 3m 58s	remaining: 1m 8s
778:	learn: 0.2462835	total: 3m 58s	remaining: 1m 7s
779:	learn: 0.2462779	total: 3m 59s	remaining: 1m 7s
780:	learn: 0.2462702	total: 3m 59s	remaining: 1m 7s
781:	learn: 0.2462648	total: 3m 59s	remaining: 1m 6s
782:	learn: 0.2462600	total: 3m 59s	remaining: 1m 6s
783:	learn: 0.2462541	total: 4m	remaining: 1m 6s
784:	learn: 0.2462457	total: 4m	remaining: 1m 5s
785:	learn: 0.2462438	total: 4m	remaining: 1m 5s
786:	learn: 0.2462386	total: 4m 1s	remaining: 1m 5s
787:	learn: 0.2462376	total: 4m 1s	remaining: 1m 4s
788:	learn: 0.2462305	total: 4m 1s	remaining: 1m 4s
789:	learn: 0.2462292	total: 4m 2s	remaining: 1m 4s
790:	learn: 0.2462233	total: 4m 2s	remaining: 1m 4s
791:	learn

929:	learn: 0.2455094	total: 4m 44s	remaining: 21.4s
930:	learn: 0.2455009	total: 4m 44s	remaining: 21.1s
931:	learn: 0.2454946	total: 4m 45s	remaining: 20.8s
932:	learn: 0.2454888	total: 4m 45s	remaining: 20.5s
933:	learn: 0.2454839	total: 4m 45s	remaining: 20.2s
934:	learn: 0.2454818	total: 4m 46s	remaining: 19.9s
935:	learn: 0.2454756	total: 4m 46s	remaining: 19.6s
936:	learn: 0.2454721	total: 4m 46s	remaining: 19.3s
937:	learn: 0.2454686	total: 4m 47s	remaining: 19s
938:	learn: 0.2454651	total: 4m 47s	remaining: 18.7s
939:	learn: 0.2454629	total: 4m 47s	remaining: 18.4s
940:	learn: 0.2454580	total: 4m 47s	remaining: 18.1s
941:	learn: 0.2454528	total: 4m 48s	remaining: 17.7s
942:	learn: 0.2454458	total: 4m 48s	remaining: 17.4s
943:	learn: 0.2454399	total: 4m 48s	remaining: 17.1s
944:	learn: 0.2454351	total: 4m 49s	remaining: 16.8s
945:	learn: 0.2454293	total: 4m 49s	remaining: 16.5s
946:	learn: 0.2454239	total: 4m 49s	remaining: 16.2s
947:	learn: 0.2454193	total: 4m 50s	remaining: 1

In [16]:
roc_auc_score(y_test, cbc.predict_proba(X_test)[:, 1])

0.9309431490315294

In [12]:
positive = target.sum()
negative = target.shape[0] - positive
weight = positive / negative

xgb = XGBClassifier(scale_pos_weight=weight)

xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=0.23084096449154373,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [13]:
roc_auc_score(y_test, xgb.predict_proba(X_test)[:, 1])

0.9311463853088515

In [22]:
# roc_auc_score(y_test, xgb.predict_proba(X_test)[:, 1]) -- for unweighted

0.9312251674833868

In [73]:
xgb.fit(train, target)

In [104]:
def make_submission(prediction, filename):
    sub['CHURN'] = prediction
    sub.to_csv('data/submissions/{}.csv'.format(filename), index=False)

In [80]:
prediction = xgb.predict_proba(full[~full['is_train']])

make_submission(prediction[:, 1], 'xgboost_baseline')

In [23]:
lr = LogisticRegression()

lr.fit(X_train, y_train)

c:\users\ivan\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [25]:
roc_auc_score(y_test, lr.predict_proba(X_test)[:, 1])

0.7260487878420422

In [81]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
import numpy as np

In [94]:
X, X_test, y, y_test = train_test_split(train, target)

space = {
    'learning_rate':    hp.uniform('learning_rate', 0.05, 0.3),
    'max_depth':        hp.choice('max_depth',        np.arange(5, 25, 1, dtype=int)),
    'min_child_weight': hp.choice('min_child_weight', np.arange(1, 10, 1, dtype=int)),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.2, 0.9),
    'subsample':        hp.uniform('subsample', 0.6, 1),
    'n_estimators':     hp.choice('n_estimators', np.arange(20, 100, 2, dtype=int)),
}

lgb_fit_params = {
    'eval_metric': 'auc',
    'early_stopping_rounds': 50,
    'verbose': False,
#     'num_iterations': 1000
}

def objective(params):
    model = LGBMClassifier(
        **params
    )
    
    X_train, X_val, y_train, y_val = train_test_split(X, y)
    
    model.fit(X_train, y_train,
              **lgb_fit_params, 
              eval_set=[(X_val, y_val)])
    
    prediction = model.predict_proba(X_test)
    score = roc_auc_score(y_test, prediction[:, 1])
    
    return -score

In [96]:
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=50)

100%|███████████████████████████████████████████████| 50/50 [11:51<00:00, 14.24s/trial, best loss: -0.9310295155359264]


In [97]:
best

{'colsample_bytree': 0.6623004625715906,
 'learning_rate': 0.13664253394809558,
 'max_depth': 8,
 'min_child_weight': 6,
 'n_estimators': 37,
 'subsample': 0.8456295422162826}

In [102]:
lgb = LGBMClassifier(**best)

lgb.fit(train.values, target.values)

LGBMClassifier(colsample_bytree=0.6623004625715906,
               learning_rate=0.13664253394809558, max_depth=8,
               min_child_weight=6, n_estimators=37,
               subsample=0.8456295422162826)

In [105]:
prediction = xgb.predict_proba(full[~full['is_train']])

make_submission(prediction[:, 1], 'lgbm_tuned')

In [113]:
target[target==1].index

Int64Index([      1,      12,      17,      21,      22,      35,      40,
                 51,      52,      54,
            ...
            2154009, 2154011, 2154012, 2154019, 2154027, 2154029, 2154033,
            2154034, 2154045, 2154047],
           dtype='int64', length=403986)

In [157]:
positive = target.sum()
negative = target.shape[0] - positive
need = negative - positive

new_idx = np.hstack([np.random.choice(target[target==1].index, need), target[target==1].index])

X_oversampled = pd.concat([train[target==0], train.loc[new_idx]])
y_oversampled = pd.concat([target[target==0], target.loc[new_idx]])

In [158]:
idx = np.random.choice(target[target==1].index, int(positive * 0.3), replace=False)

In [167]:
test_idx1 = np.random.choice(target[target==1].index, int(positive * 0.3), replace=False)
test_idx0 = np.random.choice(target[target==0].index, int(negative * 0.3), replace=False)
full_test_idx = np.hstack([test_idx0, test_idx1])
full_train_idx = np.setdiff1d(target.index, full_test_idx)

X_train, X_test, y_train, y_test = X_oversampled.loc[full_train_idx], X_oversampled.loc[full_test_idx],\
                                    y_oversampled.loc[full_train_idx], y_oversampled.loc[full_test_idx]

In [168]:
lr = LogisticRegression()

lr.fit(X_train, y_train)

roc_auc_score(y_test, lr.predict_proba(X_test)[:, 1])

c:\users\ivan\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8648446432078607

In [171]:
lgbm = LGBMClassifier(**best)

lgbm.fit(X_train, y_train)

roc_auc_score(y_test, lgbm.predict_proba(X_test)[:, 1])

0.9316109393370112

In [177]:
lgbm = LGBMClassifier(**best)

lgbm.fit(X_oversampled, y_oversampled)

LGBMClassifier(colsample_bytree=0.6623004625715906,
               learning_rate=0.13664253394809558, max_depth=8,
               min_child_weight=6, n_estimators=37,
               subsample=0.8456295422162826)

In [178]:
prediction = lgbm.predict_proba(full[~full['is_train']])

make_submission(prediction[:, 1], 'lgbm_tuned_oversample')

In [174]:
xgb = XGBClassifier()

xgb.fit(X_oversampled, y_oversampled)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [176]:
prediction = xgb.predict_proba(full[~full['is_train']])

make_submission(prediction[:, 1], 'xgb_oversample')